# New Areas of Interest

From te VIINA data
- Zaporizhzhya City (highest number of property in 2022 after Kyiv and Kharkiv)
- Dnipro (highest number of airstrikes in 2022)

In [ ]:
from shapely.geometry import box
import geopandas as gpd
from src.constants import CRS_GLOBAL
from src.data.sentinel1.download_geemap import download_s1

zapo_geo = box(34.96, 47.74, 35.3, 47.94)
dnipro_geo = box(34.8, 48.6043, 35.30186, 48.3243)
data = {"city": ["Zaporizhzhya", "Dnipro"], "geometry": [zapo_geo, dnipro_geo]}
gdf = gpd.GeoDataFrame(data, crs=CRS_GLOBAL)
gdf.explore()

In [ ]:
from src.dense_inference import extract_features, dense_inference
import joblib
from src.constants import LOGS_PATH
from pathlib import Path

aoi = 'Zaporizhzhya'
run_name = "sliding_window_train_random15"
model = joblib.load(LOGS_PATH / run_name / "model" / f"{run_name}.joblib")
folder = Path(f'./{run_name}_{aoi}')
folder.mkdir(exist_ok=True)
dense_inference(
    aoi,
    model,
    folder,
    features_extractor=extract_features,
    start_dates='2021-10-01'
)

In [ ]:
from src.data.sentinel1.time_series import get_s1_ts
from src.data import load_unosat_labels
aoi = 'UKR1'
orbit = 43
s1_ts = get_s1_ts(aoi, orbit)

In [ ]:
labels = load_unosat_labels(aoi).to_crs(s1_ts.rio.crs)
geo = labels.iloc[0].geometry
print(s1_ts.shape, geo.wkt)

In [ ]:
window_size = [3 * r for r in s1_ts.rio.resolution()]  # 3x3 pixels -> 30m
half_ws = [r // 2 for r in window_size]
ts = s1_ts.sel(
    x=slice(geo.x - half_ws[0], geo.x + half_ws[0]),
    y=slice(geo.y - half_ws[1], geo.y + half_ws[1]),
)
ts.shape

In [ ]:
ts

In [ ]:
from src.data.utils import load_ps_mask
ps_mask = load_ps_mask(aoi, orbit)
window_size = [3 * r for r in ps_mask.rio.resolution()]  # 3x3 pixels -> 30m
half_ws = [r // 2 for r in window_size]
_ps = ps_mask.sel(
    x=slice(geo.x - half_ws[0], geo.x + half_ws[0]),
    y=slice(geo.y - half_ws[1], geo.y + half_ws[1]),
)
_ps

In [ ]:
ts.attrs = {
    "aoi": aoi,
    "orbit": orbit
}

In [ ]:
ts.attrs['ps'] = _ps.values

In [ ]:
ts

In [ ]:
import h5py
import numpy as np
from tqdm import tqdm
import xarray as xr

from src.constants import AOIS_TEST, READY_PATH, TS_PATH
from src.data.utils import aoi_orbit_iterator, get_folder_ts
from src.utils.time import print_sec

extraction_strategy = 'pixel-wise-3x3'

# All time-series
fps = []
for aoi, orbit in aoi_orbit_iterator():
    folder = get_folder_ts(extraction_strategy) / aoi / f"orbit_{orbit}"
    fps += list(folder.glob("*.nc"))
print(f"Found {len(fps)} time-series")

# Create h5 file
dataset_fp = READY_PATH / f"dataset_{extraction_strategy}.h5"

In [ ]:
fp = fps[0]
ts = xr.open_dataarray(fp)
ts.stack(band_stacked=("x", "y", "band"))

In [ ]:
ts.stack(band)